In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
import cv2
import csv
from PIL import Image, ImageOps
import tensorflow as tf
from keras.models import load_model
import scipy as sp
from torchvision import transforms

In [ ]:
main_path = '/content/gdrive/My Drive/Thesis/Data/COVID-classification/'

In [ ]:
#________________Functions___________________#

# function to detect large black areas in the pictures
def find_rows_columns(im, width, height):
    rows_found=[]
    columns_found=[]
    for x in range(width):
        row = im[x, :]
        if np.mean(row) == 0 or np.mean(row) == 255:
            rows_found.append(x)
    for y in range(height):
        column = im[:,y]
        if np.mean(column) == 0 or np.mean(column) == 255:
            columns_found.append(y)
    return rows_found, columns_found

# function to reshape all images and normalize them
def reshape_normalize(image,input_path,save_path,save_folder):
    desired_size = 512
    im = Image.open(input_path+image)
    imn = ImageOps.grayscale(im)
    im = np.asarray(imn)
    # zwarte randen verwijderen
    width, height = im.shape
    rows, columns = find_rows_columns(im, width, height)
    #rijen en kolommen verwijderen
    im = np.delete(im, rows, axis=0)
    im = np.delete(im, columns, axis=1)

    #Normalizing
    im = (im-im.mean())/im.std()    #standardiseren
    im_a = (im-im.min())/(im.max()-im.min()) #Scalen tussen 0-1
    im = (255.0 / im_a.max() * (im_a - im_a.min())).astype(np.uint8) #Weer van 0-255 laten lopen en als np.uint8 type voor opslaan als png
    imn = Image.fromarray(im)

    #Resize image
    old_size = imn.size  # old_size[0] is in (width, height) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    im = imn.resize(new_size, Image.ANTIALIAS)
    new_im = Image.new("RGB",(desired_size, desired_size))   #"L" voor zwart wit
    new_im.paste(im, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))

    if image[-5] == ".jpeg":
      new_im.save(save_path+save_folder+image[:-5]+'.png')
    else:
      new_im.save(save_path+save_folder+image[:-4]+'.png')

#load in the GAN_model to make masks
model = load_model('/content/gdrive/My Drive/Thesis/GAN_model/GAN_model.h5', compile=False)

def make_masks(model,src_image_path, image, save_path):
    model = model
    src_image = Image.open(src_image_path+image)
    src_image = ImageOps.grayscale(src_image)
    src_image = np.asarray(src_image)
    src_image = src_image / 255
    src_image = np.expand_dims(src_image, 0)
    gen_msk = model.predict(src_image)

    # treshhold 
    gen_msk = np.where(gen_msk > 0.5, 1, 0)
    
    # reshape mask, add a little post_processing and save
    gen_mask = gen_msk.reshape(512, 512)
    gen_mask = sp.ndimage.binary_closing(gen_mask, structure=np.ones((25,25))).astype(np.int)
    gen_mask = sp.ndimage.binary_opening(gen_mask, structure=np.ones((15,15))).astype(np.int)
    gen_mask = sp.ndimage.binary_closing(gen_mask, structure=np.ones((20,20))).astype(np.int)
    gen_mask = sp.ndimage.binary_opening(gen_mask, structure=np.ones((10,10))).astype(np.int)
    gen_mask = sp.ndimage.binary_closing(gen_mask, structure=np.ones((15,15))).astype(np.int)
    gen_mask = sp.ndimage.binary_opening(gen_mask, structure=np.ones((10,10))).astype(np.int)
    gen_mask = sp.ndimage.binary_closing(gen_mask, structure=np.ones((10,10))).astype(np.int)
    gen_mask = sp.ndimage.binary_opening(gen_mask, structure=np.ones((10,10))).astype(np.int)
    gen_mask = sp.ndimage.binary_closing(gen_mask, structure=np.ones((5,5))).astype(np.int)
    gen_mask = sp.ndimage.binary_opening(gen_mask, structure=np.ones((3,3))).astype(np.int)
    gen_mask = Image.fromarray((gen_mask * 255).astype(np.uint8))
    gen_mask.save(save_path+image)

# Function to lay the masks over the original images
def combine(image_path,mask_path,image,names,save_combined):
    image = np.asarray(Image.open(image_path+image))
    mask = Image.open(mask_path+names[idx])
    mask = np.expand_dims(mask,2)
    mask = np.asarray(mask)
    combined = np.multiply(image,mask)
    combined = Image.fromarray(combined)
    combined.save(save_combined+names[idx])

In [ ]:
directory_files = os.listdir(main_path+'covid/') 
COVID_names = [file for file in directory_files if file.endswith('.png') or file.endswith('.jpg') or file.endswith('.jpeg')]
COVID_names.sort()

directory_files = os.listdir(main_path+'non_covid/') 
non_cov_names = [file for file in directory_files if file.endswith('.png') or file.endswith('.jpg') or file.endswith('.jpeg')]
non_cov_names.sort()

print(len(COVID_names), COVID_names[:1], len(non_cov_names), non_cov_names[:1])

4898 ['0001-56222.png'] 4923 ['00000003_007.png']


In [ ]:
non_covid_folder = 'non_covid/'
input_path = main_path+'geen_cov/'

with tf.device('/device:GPU:0'):
  for image in non_cov_names:
    reshape_normalize(image,input_path,main_path,non_covid_folder)

In [ ]:
# using the GAN model to create masks

covid_path = main_path+'covid/'
save_mask_covid = main_path+'masks/covid/'
non_covid_path = main_path+'non_covid/'
save_mask_non_covid = main_path+'masks/non_covid/'

with tf.device('/device:GPU:0'):
  for image in (COVID_names):
    make_masks(model,covid_path,image,save_mask_covid)
  
  for image in (non_cov_names):
    make_masks(model,non_covid_path,image,save_mask_non_covid)

In [ ]:
# laying the masks over the covid images

image_path = main_path+'covid/'
mask_path = main_path+'masks/covid/'
names = COVID_names
save_combined = main_path+'covid_with_masks/'

with tf.device('/device:GPU:0'):
  for idx, image in enumerate(names):
    combine(image_path,mask_path,image,names,save_combined)

In [ ]:
# laying the masks over the non_covid images

image_path = main_path+'non_covid/'
mask_path = main_path+'masks/non_covid/'
names = non_cov_names
save_combined = main_path+'non_covid_with_masks/'

with tf.device('/device:GPU:0'):
  for idx, image in enumerate(names):
    combine(image_path,mask_path,image,names,save_combined)

In [ ]:
# resize everyting from 512*512 to 256*256

In [ ]:
# making the train-test split

#covid
with tf.device('/device:GPU:0'):
  for idx, image in enumerate(COVID_names):
    im = Image.open(main_path+'covid/'+image)
    im_m = Image.open(main_path+'covid_with_masks/'+image)
    if idx % 5 == 0:
      im.save(main_path+'test/covid/'+image)
      im_m.save(main_path+'test_masks/covid/'+image)
    else:
      im.save(main_path+'train/covid/'+image)
      im_m.save(main_path+'train_masks/covid/'+image)

#non_covid
  for idx, image in enumerate(non_cov_names):
    im = Image.open(main_path+'non_covid/'+image)
    im_m = Image.open(main_path+'non_covid_with_masks/'+image)
    if idx % 5 == 0:
      im.save(main_path+'test/non_covid/'+image)
      im_m.save(main_path+'test_masks/non_covid/'+image)
    else:
      im.save(main_path+'train/non_covid/'+image)
      im_m.save(main_path+'train_masks/non_covid/'+image)

In [ ]:
directory_files = os.listdir(main_path+'train/covid/') 
trn_cov = [file for file in directory_files if file.endswith('.png')]
directory_files = os.listdir(main_path+'train/non_covid/') 
trn_non_cov = [file for file in directory_files if file.endswith('.png')]

#making validation sets
with tf.device('/device:GPU:0'):  
  for idx, image in enumerate(trn_cov):
    if idx % 10 == 0:
      im = Image.open(main_path+'train/covid/'+image)
      im_m = Image.open(main_path+'train_masks/covid/'+image)
      im.save(main_path+'val/covid/'+image)
      im_m.save(main_path+'val_masks/covid/'+image)
      os.remove(main_path+'train/covid/'+image)
      os.remove(main_path+'train_masks/covid/'+image)

  for idx, image in enumerate(trn_non_cov):
    if idx % 10 == 0:
      im = Image.open(main_path+'train/non_covid/'+image)
      im_m = Image.open(main_path+'train_masks/non_covid/'+image)
      im.save(main_path+'val/non_covid/'+image)
      im_m.save(main_path+'val_masks/non_covid/'+image)
      os.remove(main_path+'train/non_covid/'+image)
      os.remove(main_path+'train_masks/non_covid/'+image)